# 1D Heisenberg J1J2 model: DMRG

In this notebook, we use DMRG to calculate the ground state energies of the 1D Heisenberg J1J2 model with N=50 and N=100 spins, with J1 = 1.0 and J2 = 0.0, 0.2, 0.5, 0.8. 

Some useful links:
- https://en.wikipedia.org/wiki/J1_J2_model
- https://en.wikipedia.org/wiki/Quantum_Heisenberg_model
- https://en.wikipedia.org/wiki/Density_matrix_renormalization_group#Example:_Quantum_Heisenberg_model
- https://tenpy.readthedocs.io/en/latest/reference/tenpy.models.html
- https://tenpy.readthedocs.io/en/latest/intro/model.html
- https://tenpy.johannes-hauschild.de/viewtopic.php?t=446

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tenpy.models.spins import SpinModel
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg
from tenpy.models.lattice import Chain
from tenpy.networks.site import SpinHalfSite
from tenpy.models.model import CouplingMPOModel

https://tenpy.readthedocs.io/en/latest/reference/tenpy.models.model.CouplingMPOModel.html

In [4]:
def define_model_params(N,j1,j2, bc_MPSS, bc_xx):
    #bc_xx is either 'open' or 'periodic'
    #bc_MPSS is either 'finite' or 'infinite'
    model_params = {
                'conserve': None,
                'Lx': N,
                'J1': j1,
                'J2': j2,
                'bc_MPS': bc_MPSS, 
                'bc_x': bc_xx, 
                }
    return model_params

In [5]:
class MyModel(CouplingMPOModel):     
    def init_sites(self, model_params):
        site = SpinHalfSite(conserve='None')
        return site
    
    def init_lattice(self, model_params):
        Lx =  model_params.get('Lx',10)
        bc_x = model_params.get('bc_x','open')
        bc_MPS = model_params.get('bc_MPS', 'finite')
        lattice = Chain(Lx, site=self.init_sites(model_params), bc_MPS=bc_MPS)
        return lattice
    def init_terms(self, model_params):
        J1 = model_params.get('J1', 1.)
        J2 = model_params.get('J2', 0.2)
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(J1, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J1, u1, 'Sy', u2, 'Sy', dx)
            self.add_coupling(J1, u1, 'Sx', u2, 'Sx', dx)
        for u1, u2, dx in self.lat.pairs['next_nearest_neighbors']:
            self.add_coupling(J2, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J2, u1, 'Sy', u2, 'Sy', dx)
            self.add_coupling(J2, u1, 'Sx', u2, 'Sx', dx)


In [6]:
dmrg_params = {
    'mixer': True,  
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    'verbose': True,
}


In [5]:
#test_model = MyModel(model_params)
#test_model.init_lattice(model_params)
#test_model.lat.pairs['nearest_neighbors']

In [7]:
def run_dmrg(N, j1, j2, bc_MPSS, bc_xx):
    model_params = define_model_params(N, j1, j2, bc_MPSS, bc_xx)
    M = MyModel(model_params)
    lattice = M.init_lattice(model_params)
    initial_state = ['up'] * N # initial state
    psi = MPS.from_product_state(lattice.mps_sites(), initial_state, bc=lattice.bc_MPS)

    eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
    E,psi = eng.run() # the main work; modifies psi in place
    return E, psi

# N=50

In [8]:
E50, psi50 = run_dmrg(50,1.,0.0, 'finite', 'open')
print("ground state energy = ", E50)

ground state energy =  -21.972110281116198


In [9]:
E50, psi50 = run_dmrg(50,1.,0.2, 'finite', 'open')
print("ground state energy = ", E50)

ground state energy =  -20.314982335144908


In [10]:
E50, psi50 = run_dmrg(50,1.,0.5, 'finite', 'open')
print("ground state energy = ", E50)

ground state energy =  -18.75000000000001


In [11]:
E50, psi50 = run_dmrg(50,1.,0.8, 'finite', 'open')
print("ground state energy = ", E50)

ground state energy =  -20.98414964797433


## N=100

In [12]:
E100, psi100 = run_dmrg(100,1.,0.2, 'finite', 'open')
print("ground state energy = ", E100)

ground state energy =  -40.73881904063248


In [13]:
E100b, psi100b = run_dmrg(100,1.,0.5, 'finite', 'open')
print("ground state energy = ", E100b)

ground state energy =  -37.50000000000013


In [14]:
E100c, psi100c = run_dmrg(100,1.,0., 'finite', 'open')
print("ground state energy = ", E100c)

ground state energy =  -44.12773986967251


In [15]:
E100d, psi100d = run_dmrg(100,1.,0.8, 'finite', 'open')
print("ground state energy = ", E100d)

ground state energy =  -42.07006297371643
